In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input/facial-expression/fer2013/"))

# Any results you write to the current directory are saved as output.

['fer2013.csv']


In [2]:
import tensorflow as tf

import keras
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, AveragePooling2D
from keras.layers import Dense, Activation, Dropout, Flatten

from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

Using TensorFlow backend.


In [3]:
# get the data
filname = '../input/facial-expression/fer2013/fer2013.csv'
label_map = ['Anger', 'Disgust', 'Fear', 'Happy', 'Sad', 'Surprise', 'Neutral']
names=['emotion','pixels','usage']

# dataset source: https://www.kaggle.com/ahmedmoorsy/facial-expression (image sudah di convert ke np array)
df=pd.read_csv('../input/facial-expression/fer2013/fer2013.csv',names=names, na_filter=False)
im=df['pixels']
df.head(10)

,emotion,pixels,usage
0,emotion,pixels,Usage
1,0,70 80 82 72 58 58 60 63 54 58 60 48 89 115 121...,Training
2,0,151 150 147 155 148 133 111 140 170 174 182 15...,Training
3,2,231 212 156 164 174 138 161 173 182 200 106 38...,Training
4,4,24 32 36 30 32 23 19 20 30 41 21 22 32 34 21 1...,Training
5,6,4 0 0 0 0 0 0 0 0 0 0 0 3 15 23 28 48 50 58 84...,Training
6,2,55 55 55 55 55 54 60 68 54 85 151 163 170 179 ...,Training
7,4,20 17 19 21 25 38 42 42 46 54 56 62 63 66 82 1...,Training
8,3,77 78 79 79 78 75 60 55 47 48 58 73 77 79 57 5...,Training
9,3,85 84 90 121 101 102 133 153 153 169 177 189 1...,Training


In [4]:
def getData(filname):
    # images are 48x48
    # N = 35887
    Y = []
    X = []
    X_train = []
    X_test = []
    y_train = []
    y_test = []
    first = True
    for line in open(filname):
        if first:
            first = False
        else:
            row = line.split(',')
            if (int(row[0]))==1: continue
            if (int(row[0]))>1: 
                Y.append(int(row[0])-1)
                ykecil = int(row[0])-1;
            else : 
                Y.append(int(row[0]))
                ykecil = int(row[0])
            X.append([int(p) for p in row[1].split()])
            
            #hapus aja 
#             print(row)
            ytemp=ykecil
            if ytemp==5: ykecil =3
            if ytemp==3: ykecil =4
            if ytemp==4: ykecil = 5
            if row[2]=='Training\n':
                X_train.append([int(p) for p in row[1].split()])
                y_train.append(ykecil)
            else :
                X_test.append([int(p) for p in row[1].split()])
                y_test.append(ykecil)

#     X, Y = np.array(X) / 255.0, np.array(Y)
#     return X, Y

    #hapus aja
    X_train, X_test, y_train, y_test = np.array(X_train)/255.0, np.array(X_test)/255.0, np.array(y_train), np.array(y_test)
    return X_train, X_test, y_train, y_test


In [5]:
num_class = 6
X_train, X_test, y_train, y_test = getData(filname)
print(X_train.shape)
print(X_train)
print(y_train)
print(np.unique(y_train, return_counts=True))
N, D = X_train.shape
X_train = X_train.reshape(N, 48, 48, 1)
N, D = X_test.shape
X_test = X_test.reshape(N, 48, 48, 1)
y_train = (np.arange(num_class) == y_train[:, None]).astype(np.float32)
y_test = (np.arange(num_class) == y_test[:, None]).astype(np.float32)

(28273, 2304)
[[0.2745098  0.31372549 0.32156863 ... 0.41568627 0.42745098 0.32156863]
 [0.59215686 0.58823529 0.57647059 ... 0.75686275 0.71764706 0.72156863]
 [0.90588235 0.83137255 0.61176471 ... 0.34509804 0.43137255 0.59607843]
 ...
 [0.29019608 0.31764706 0.34117647 ... 0.7372549  0.73333333 0.73333333]
 [0.87058824 0.89019608 0.79607843 ... 0.53333333 0.53333333 0.5254902 ]
 [0.76470588 0.78039216 0.80392157 ... 0.02352941 0.05882353 0.14901961]]
[0 0 1 ... 4 0 4]
(array([0, 1, 2, 3, 4, 5]), array([3995, 4097, 7215, 4965, 4830, 3171]))


In [6]:
print(np.unique(y_train, return_counts=True))
print(y_train)

(array([0., 1.], dtype=float32), array([141365,  28273]))
[[1. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0.]
 ...
 [0. 0. 0. 0. 1. 0.]
 [1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0.]]


In [7]:
from keras.models import Sequential
from keras.layers import Dense , Activation , Dropout ,Flatten
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.metrics import categorical_accuracy
from keras.models import model_from_json
from keras.callbacks import ModelCheckpoint
from keras.optimizers import *
from keras.layers.normalization import BatchNormalization

In [8]:
def my_model():
    model = Sequential()
    input_shape = (48,48,1)
    num_features=64
    
    model.add(Conv2D(64, (5, 5), input_shape=input_shape,activation='relu', padding='same'))
    model.add(Conv2D(64, (5, 5), activation='relu', padding='same'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Conv2D(128, (5, 5),activation='relu',padding='same'))
    model.add(Conv2D(128, (5, 5),activation='relu',padding='same'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Conv2D(256, (3, 3),activation='relu',padding='same'))
    model.add(Conv2D(256, (3, 3),activation='relu',padding='same'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2, 2)))
    
    model.add(Conv2D(512, (3, 3),activation='relu',padding='same'))
    model.add(Conv2D(512, (3, 3),activation='relu',padding='same'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Flatten())
    model.add(Dense(1024))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(Dropout(0.2))
    
    model.add(Dense(512))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(Dropout(0.2))
    
    model.add(Dense(num_class))
    model.add(Activation('softmax'))
    
    model.compile(loss='categorical_crossentropy', metrics=['accuracy'],optimizer='nadam')
    
    return model

In [9]:
model=my_model()
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 48, 48, 64)        1664      
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 48, 48, 64)        102464    
_________________________________________________________________
batch_normalization_1 (Batch (None, 48, 48, 64)        256       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 24, 24, 64)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 24, 24, 128)       204928    
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 24, 24, 128)       409728    
_________________________________________________________________
batch_normalization_2 (Batch (None, 24, 24, 128)       512       
__________

In [ ]:
path_model='model_filter.h5'
K.tensorflow_backend.clear_session()
model=my_model()
K.set_value(model.optimizer.lr,1e-3)

callback = tf.keras.callbacks.EarlyStopping(monitor='val_acc', patience=5)
h=model.fit(x=X_train,     
            y=y_train, 
            batch_size=64, 
            epochs=100, 
            callbacks=[callback],
            validation_split=0.2,
            shuffle=True
            )

Train on 22618 samples, validate on 5655 samples
Epoch 1/100
22618/22618 [==============================] - 21s 946us/step - loss: 1.6842 - acc: 0.3312 - val_loss: 1.9182 - val_acc: 0.3943
Epoch 2/100
22618/22618 [==============================] - 14s 627us/step - loss: 1.3301 - acc: 0.4767 - val_loss: 1.2868 - val_acc: 0.5026
Epoch 3/100
22618/22618 [==============================] - 14s 636us/step - loss: 1.1626 - acc: 0.5500 - val_loss: 1.1984 - val_acc: 0.5307
Epoch 4/100
22618/22618 [==============================] - 14s 628us/step - loss: 1.0513 - acc: 0.5971 - val_loss: 1.1619 - val_acc: 0.5540
Epoch 5/100
22618/22618 [==============================] - 14s 634us/step - loss: 0.9671 - acc: 0.6333 - val_loss: 1.2072 - val_acc: 0.5236
Epoch 6/100
22618/22618 [==============================] - 14s 628us/step - loss: 0.8728 - acc: 0.6705 - val_loss: 1.3309 - val_acc: 0.5307
Epoch 7/100
22618/22618 [==============================] - 14s 638us/step - loss: 0.7647 - acc: 0.7162 - val_lo

In [ ]:
loss,accuracy = model.evaluate(X_test, y_test)
print(loss)
print(accuracy*100)

In [ ]:
import seaborn as sn
from sklearn.metrics import confusion_matrix
import pandas as pd

y_pred=model.predict(X_test)
y_true=y_test
classes = ['Anger', 'Fear', 'Happy', 'Sad', 'Surprise', 'Neutral']

y_pred=np.argmax(y_pred,axis=1)
y_true=np.argmax(y_true,axis=1)

for i in range (len(y_pred)):
    if i>20: break
    print(y_pred[i])

print("haha")
for i in range (len(y_true)):
    
    print(y_true[i])
    if(i>20): break
        

print(y_pred)
print(y_true)

cf_matrix=confusion_matrix(y_true=y_true, y_pred=y_pred)

print(cf_matrix)

df_cm = pd.DataFrame(cf_matrix, index = ['Anger', 'Fear', 'Happy', 'Neutral', 'Sad', 'Surprise'], columns = ['Anger', 'Fear', 'Happy', 'Neutral', 'Sad', 'Surprise'])

plt.figure(figsize = (10,7))
sn.heatmap(df_cm, annot=True, fmt='d')